In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


In [19]:
url_data = "https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=BlocksFavour&agg=PerGame"
years = list(range(2001, 2022))

In [20]:

for year in years:
    with open("blocks_leaders_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [21]:
dfs = []
for year in years :
    with open ("steals_leaders_htmls/{}.html".format(year),encoding="utf-8") as f :
        page =f.read()
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr',class_="StatsHeaderGridResults")
        statistics_table=soup.find(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")
        blocks_data= pd.read_html(str(statistics_table))[0]
        blocks_data["Year"] = year
        dfs.append(blocks_data)
                

In [22]:
blocks_data = pd.concat(dfs)
blocks_data

,Rank,Player,Team,GamesG,StealsST,Ave.,Per 40 min.40m,Year
0,1,"GINOBILI, EMANUELGINOBILI, E.",VIRKinder Virtus BolognaKinder Virtus Bologna,22,58,2.64,3.70,2001
1,2,"HUGGINS, ROGERHUGGINS, R.",CHASpirou BasketSpirou Basket,14,32,2.29,3.07,2001
2,3,"JARIC, MARKOJARIC, M.",VIRKinder Virtus BolognaKinder Virtus Bologna,21,47,2.24,3.17,2001
3,4,"PHELPS, DERRICKPHELPS, D.",BERAlba BerlinAlba Berlin,13,29,2.23,2.75,2001
4,5,"BIGGS, RALPHBIGGS, R.",OOSTelindus OostendeTelindus Oostende,14,30,2.14,2.33,2001
...,...,...,...,...,...,...,...,...
25,25,"GIEDRAITIS, ROKASGIEDRAITIS, R.",BKNBitci Baskonia Vitoria-GasteizBitci Baskoni...,29,29,1.00,1.39,2021
26,25,"BACON, DWAYNEBACON, D.",ASMAS MonacoAS Monaco,29,29,1.00,1.58,2021
27,25,"WALDEN, COREYWALDEN, C.",BAYFC Bayern MunichFC Bayern Munich,25,25,1.00,1.79,2021
28,25,"JEKIRI, TONYEJEKIRI, T.",UNKUNICS KazanUNICS Kazan,24,24,1.00,1.63,2021


In [9]:
engine = create_engine('sqlite:///blocks_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [10]:
blocks_data.to_sql(
    name = "blocks_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [11]:
blocks_data_select = session.execute('select * from "blocks_leaders"')
blocks_data_select.all()

[(0, 1, 'DUNSTON, BRYANTDUNSTON, B.', 'EFSAnadolu Efes IstanbulAnadolu Efes Istanbul', 30, 52, 1.73, 2.46, 2001),
 (1, 2, 'TAVARES, WALTERTAVARES, W.', 'RMBReal MadridReal Madrid', 29, 43, 1.48, 3.4, 2001),
 (2, 3, 'PLEISS, TIBORPLEISS, T.', 'VBCValencia BasketValencia Basket', 29, 32, 1.1, 2.29, 2001),
 (3, 4, 'PARAKHOUSKI, ARTSIOMPARAKHOUSKI, A.', 'MTAMaccabi FOX Tel AvivMaccabi FOX Tel Aviv', 30, 31, 1.03, 2.55, 2001),
 (4, 4, 'GUDAITIS, ARTURASGUDAITIS, A.', 'AXMAX Armani Exchange Olimpia MilanAX Armani Exchange Olimpia Milan', 29, 31, 1.07, 2.07, 2001),
 (5, 6, 'TYUS, ALEXTYUS, A.', 'MTAMaccabi FOX Tel AvivMaccabi FOX Tel Aviv', 30, 30, 1.0, 2.0, 2001),
 (6, 7, 'POIRIER, VINCENTPOIRIER, V.', 'BKNKIROLBET Baskonia Vitoria GasteizKIROLBET Baskonia Vitoria Gasteiz', 34, 29, 0.85, 1.91, 2001),
 (7, 8, 'BOLDEN, JONAHBOLDEN, J.', 'MTAMaccabi FOX Tel AvivMaccabi FOX Tel Aviv', 29, 27, 0.93, 1.76, 2001),
 (8, 9, 'SINGLETON, CHRISSINGLETON, C.', 'PAOPanathinaikos Superfoods AthensPanathina